# Lesson 1: Building Your First Agent with Google's ADK

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:

1. **Understand** the core concepts of Google's Agent Development Kit (ADK)
2. **Configure** the ADK environment for use with OpenAI API
3. **Create** a simple AI agent with custom instructions and personality
4. **Interact** with your agent using the Runner and session management
5. **Modify** agent behavior to suit different use cases

## 📚 Context

Welcome to the first lesson in our ADK series! In the previous notebooks, you learned about the **Model Context Protocol (MCP)**, which enables AI models to interact with external tools and data sources.

The **Agent Development Kit (ADK)** builds on these concepts by providing a complete framework for creating sophisticated AI agents that can:
- Maintain conversation context across multiple turns
- Use tools to perform actions
- Work together in multi-agent systems
- Integrate with MCP servers


Throughout this series, we'll be building agents for an **IT Support Services Company**. In this first lesson, we'll create a simple support agent that can help users with basic IT questions.

---

## 🔧 Part 1: Environment Setup

Let's start by installing the required packages and configuring our environment for Google Colab.

In [ ]:
# Install the Google Agent Development Kit and dependencies
# This may take a minute or two
!pip install -q google-adk litellm openai python-dotenv nest-asyncio

print("✅ Packages installed successfully!")

### Import Required Libraries

Now let's import the necessary modules from the ADK framework.

In [ ]:
# Core ADK imports
from google.adk.agents import LlmAgent  # For creating AI agents
from google.adk.runners import Runner  # For executing agent interactions
from google.adk.sessions import InMemorySessionService  # For managing conversation sessions
from google.adk.models.lite_llm import LiteLlm  # For using OpenAI and other LLM providers

# Google AI SDK for content formatting
from google.genai import types

# System imports
import os
import asyncio

print("✅ Imports successful!")

### Configure OpenAI API Authentication

**Important:** To use ADK with OpenAI, you need:
1. An OpenAI API key from [OpenAI Platform](https://platform.openai.com/api-keys)
2. Set the environment variable to configure the API key

**Note:** In a production environment, you would store API keys securely. For this tutorial, we'll use Colab's userdata feature or direct input.

In [ ]:
# Configure OpenAI API key
# Method 1: Try to get API key from Colab secrets (recommended)
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Method 2: Manual input (fallback)
    from getpass import getpass
    print("💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Validate that the API key is set
if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == "":
    raise ValueError(
        "❌ ERROR: No API key provided!\n"
        "Please get your API key from: https://platform.openai.com/api-keys"
    )

print("✅ Authentication configured successfully!")
print(f"🔐 API Key (first 10 chars): {OPENAI_API_KEY[:10]}...")

# Configure which OpenAI model to use
# Options: "gpt-4o", "gpt-4o-mini", "gpt-4-turbo", "gpt-3.5-turbo", "gpt-5-nano", etc.
OPENAI_MODEL = "gpt-5-nano"  # Using gpt-5-nano for cost efficiency
print(f"🤖 Selected Model: {OPENAI_MODEL}")

---

## 🏗️ Part 2: Understanding ADK Architecture

Before we build our first agent, let's understand the key components of the ADK framework:

### Core Components

1. **Agent (`LlmAgent`)**
   - The AI brain that processes user requests
   - Configured with a specific LLM model (e.g., `gpt-5-nano`)
   - Has instructions that define its behavior and personality
   - Can use tools to perform actions (we'll cover this in the following notebook)

2. **Runner**
   - Manages the execution of agent interactions
   - Handles the flow of messages between user and agent
   - Coordinates with session and artifact services

3. **Session Service (`InMemorySessionService`)**
   - Stores conversation history and context
   - Enables agents to "remember" previous messages
   - `InMemorySessionService` stores data in RAM (resets when notebook restarts)
   - In production, you'd use persistent storage (e.g., Firestore)

### How They Work Together

```
User Message → Runner → Agent (processes with LLM) → Runner → Response
                ↓                                      ↑
         Session Service (stores conversation history)
```



- **Agents** can maintain context across multiple conversation turns
- **Runners** provide a consistent interface for agent interaction
- **Sessions** enable personalized, stateful conversations

Now let's build our first agent!

---

## 🤖 Part 3: Creating Your First Agent

Let's create a simple IT support agent. This agent will have a helpful personality and be knowledgeable about basic IT support topics.

### Agent Configuration Parameters

- **`model`**: The LLM to use (we'll use OpenAI's `gpt-5-nano` - fast and efficient)
- **`name`**: Identifier for the agent (useful in multi-agent systems)
- **`instruction`**: System prompt that defines the agent's behavior and personality
- **`tools`**: Functions the agent can call (we'll add these in the following notebook)


In [ ]:
# Create an IT Support Agent using OpenAI
it_support_agent = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),  # Use OpenAI model via LiteLLM
    name="it_support_assistant",  # Unique identifier for this agent
    instruction="""
    You are Alex, a friendly and knowledgeable IT Support Assistant for TechHelp Solutions,
    a university IT support services company.

    Your role:
    - Help students and faculty with common IT issues
    - Provide clear, step-by-step instructions
    - Be patient and encouraging, especially with users who are not tech-savvy
    - If you don't know something, admit it and suggest they contact human support

    Your personality:
    - Friendly and approachable
    - Professional but not overly formal
    - Use simple language and avoid unnecessary jargon
    - Add occasional light humor to make users feel comfortable

    Remember: You're here to help, not to show off technical knowledge.
    Your goal is to solve problems and make users feel supported.
    """
)

print("✅ IT Support Agent created successfully!")
print(f"📋 Agent Name: {it_support_agent.name}")
print(f"🤖 Model: OpenAI {OPENAI_MODEL}")

### Set Up Session Management

Now we'll create a session service to manage conversation history.

In [ ]:
# Create an in-memory session service
# This will store conversation history while the notebook is running
session_service = InMemorySessionService()

print("✅ Session service initialized!")
print("💡 Note: Session data is stored in memory and will be lost when the notebook restarts.")

### Create the Runner

The Runner coordinates the agent's execution and manages sessions.

In [ ]:
# Configuration for our ADK application
APP_NAME = "it_support_app"  # Unique identifier for our application

# Create the runner with our agent and session service
runner = Runner(
    app_name=APP_NAME,
    agent=it_support_agent,
    session_service=session_service
)

print("✅ Runner initialized and ready!")
print(f"📱 App Name: {APP_NAME}")

---

## 💬 Part 4: Interacting with Your Agent

Now comes the exciting part - let's talk to our agent!

### How Agent Interaction Works

1. Create a user message using the `types.Content` format
2. Send it to the runner with a session ID and user ID
3. The runner processes the message through the agent
4. Receive a stream of events (including the final response)

### Helper Function

Let's create a convenient function to interact with our agent:

In [ ]:
# Track created sessions to avoid recreating them
_created_sessions = set()

async def chat_with_agent_async(user_message: str, session_id: str = "session_001", user_id: str = "student_001"):
    """
    Send a message to the IT support agent and print the response (async version).

    Args:
        user_message: The message to send to the agent
        session_id: Unique identifier for the conversation session
        user_id: Unique identifier for the user
    """
    # Create the session if it doesn't exist yet
    # This is required by InMemorySessionService before the runner can use it
    session_key = (session_id, user_id)
    if session_key not in _created_sessions:
        await session_service.create_session(
            app_name=APP_NAME,
            user_id=user_id,
            session_id=session_id,
            state={}  # Initial empty state
        )
        _created_sessions.add(session_key)

    # Format the user message in the required Content structure
    content = types.Content(
        role='user',
        parts=[types.Part(text=user_message)]
    )

    # Send the message to the runner and get events (using async version)
    events = runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=content
    )

    # Process events to find the final response
    print(f"\n👤 You: {user_message}")
    print(f"\n🤖 Alex: ", end="")

    async for event in events:
        # Check if this is the final response event
        if event.is_final_response():
            # Extract and print the agent's response
            final_response = event.content.parts[0].text
            print(final_response)
            return final_response

    print("\n⚠️ No response received from agent.")
    return None

# Wrapper function for easier use in Colab (handles async automatically)
def chat_with_agent(user_message: str, session_id: str = "session_001", user_id: str = "student_001"):
    """
    Send a message to the IT support agent and print the response.

    This is a convenience wrapper that handles async execution automatically.

    Args:
        user_message: The message to send to the agent
        session_id: Unique identifier for the conversation session
        user_id: Unique identifier for the user
    """
    # Check if we're already in an event loop (Colab/Jupyter context)
    try:
        loop = asyncio.get_running_loop()
        # We're in an async context, need to use nest_asyncio or create task
        import nest_asyncio
        nest_asyncio.apply()
        return asyncio.run(chat_with_agent_async(user_message, session_id, user_id))
    except RuntimeError:
        # No event loop running, we can use asyncio.run directly
        return asyncio.run(chat_with_agent_async(user_message, session_id, user_id))

print("✅ Helper function created!")

### Test Your Agent

Let's have our first conversation with the IT support agent!

In [ ]:
# First interaction - greeting
chat_with_agent("Hi! I'm having trouble connecting to the university Wi-Fi.")

In [ ]:
# Follow-up question - the agent should remember the context from the previous message
chat_with_agent("It's an iPhone. What should I do first?")

In [ ]:
# Another topic
chat_with_agent("Thanks! Also, how do I reset my university email password?")

### Testing Session Memory

Let's verify that our agent maintains context within a session but starts fresh in a new session.

In [ ]:
# Start a new session with a different session_id
print("\n" + "="*60)
print("Starting a NEW SESSION (the agent won't remember previous messages)")
print("="*60)

chat_with_agent(
    "What did we talk about before?",
    session_id="session_002"  # Different session ID
)

### Interactive Chat (Try It Yourself!)

Now it's your turn! Try asking the agent different IT support questions:

In [ ]:
# Type your own message here and run this cell
your_message = "How do I print documents from the library?"  # Change this to your own question!

chat_with_agent(your_message, session_id="my_session")

---

## 🎓 Part 5: Exercises

Now it's time to practice! Complete these exercises to reinforce your learning.

### Exercise 1: Modify the Agent's Personality (Beginner)

**Task:** Create a new agent with a different personality. Instead of "Alex the friendly helper", create "Dr. Tech" who is more formal and academic in tone.

**Instructions:**
1. Copy the agent creation code below
2. Modify the `instruction` parameter to change the personality
3. Test your new agent with a few questions

**Hint:** Think about how a formal academic advisor would communicate vs. a friendly peer.

In [ ]:
# Exercise 1: Create your custom agent here

# TODO: Create a new LlmAgent with a formal, academic personality
dr_tech_agent = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="dr_tech",
    instruction="""
    # YOUR CODE HERE: Write instructions for a formal, academic IT support persona
    # Think about:
    # - How should Dr. Tech introduce themselves?
    # - What tone should they use?
    # - How formal should the language be?
    """
)

# Create a new runner for this agent
dr_tech_runner = Runner(
    app_name="dr_tech_app",
    agent=dr_tech_agent,
    session_service=session_service
)

# Track sessions for Dr. Tech
_dr_tech_sessions = set()

# Async version of the helper function
async def chat_with_dr_tech_async(user_message: str, session_id: str = "drtech_session", user_id: str = "student_001"):
    # Create session if it doesn't exist
    session_key = (session_id, user_id)
    if session_key not in _dr_tech_sessions:
        await session_service.create_session(
            app_name="dr_tech_app",
            user_id=user_id,
            session_id=session_id,
            state={}
        )
        _dr_tech_sessions.add(session_key)

    content = types.Content(role='user', parts=[types.Part(text=user_message)])
    events = dr_tech_runner.run_async(user_id=user_id, session_id=session_id, new_message=content)

    print(f"\n👤 You: {user_message}")
    print(f"\n🎓 Dr. Tech: ", end="")

    async for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print(final_response)
            return final_response

# Synchronous wrapper
def chat_with_dr_tech(user_message: str, session_id: str = "drtech_session", user_id: str = "student_001"):
    try:
        loop = asyncio.get_running_loop()
        import nest_asyncio
        nest_asyncio.apply()
        return asyncio.run(chat_with_dr_tech_async(user_message, session_id, user_id))
    except RuntimeError:
        return asyncio.run(chat_with_dr_tech_async(user_message, session_id, user_id))

# Test your new agent
# chat_with_dr_tech("Hi! I need help with my Wi-Fi connection.")

### Exercise 2: Specialized Support Agent (Intermediate)

**Task:** Create a specialized agent that only helps with **printer and printing issues**. The agent should politely redirect users if they ask about other topics.

**Instructions:**
1. Create a new agent with instructions that specialize in printing support
2. The agent should:
   - Be expert in printer-related issues
   - Politely redirect non-printing questions to general IT support
3. Test with both printing and non-printing questions

**Challenge:** Make the agent remember if a user asks about printing in one message, then asks a related follow-up.

In [ ]:
# Exercise 2: Create a specialized printer support agent

# TODO: Create your specialized agent here
printer_agent = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="printer_specialist",
    instruction="""
    # YOUR CODE HERE: Write instructions for a printer-specialized support agent
    # The agent should:
    # 1. Be expert in printer/printing issues
    # 2. Politely decline non-printing questions
    # 3. Redirect to general support when needed
    """
)

# TODO: Create runner and test function
# printer_runner = Runner(...)

# TODO: Test with multiple questions:
# 1. A printing question (should help)
# 2. A Wi-Fi question (should redirect)
# 3. A follow-up printing question (should remember context)

### Exercise 3: Multi-Turn Conversation Flow (Advanced)

**Task:** Create a troubleshooting agent that guides users through a structured diagnostic process.

**Instructions:**
1. Create an agent that asks diagnostic questions step-by-step
2. The agent should:
   - Start by asking about the device type
   - Then ask about what error message they see
   - Finally provide a solution based on the answers
3. Write a conversation script that demonstrates this flow

**Challenge:** Can you make the agent remember all the diagnostic information collected across multiple messages?

In [ ]:
# Exercise 3: Create a diagnostic troubleshooting agent

# TODO: Create your diagnostic agent
diagnostic_agent = LlmAgent(
    model=LiteLlm(model=f"openai/{OPENAI_MODEL}"),
    name="diagnostic_troubleshooter",
    instruction="""
    # YOUR CODE HERE: Write instructions for a step-by-step diagnostic agent
    # The agent should:
    # 1. Ask about device type first
    # 2. Ask about specific symptoms/errors
    # 3. Collect information systematically
    # 4. Provide solutions based on gathered information
    """
)

# TODO: Create runner
# diagnostic_runner = Runner(...)

# TODO: Create a multi-turn conversation that demonstrates:
# Message 1: User states they have a problem
# Message 2: User answers the device question
# Message 3: User describes the error
# Message 4: Agent provides solution based on context

# Example conversation flow:
# chat_with_diagnostic("I can't access my email", session_id="diagnostic_001")
# chat_with_diagnostic("I'm using a laptop", session_id="diagnostic_001")
# chat_with_diagnostic("It says 'incorrect password' but I know it's right", session_id="diagnostic_001")

---

## 🎯 Part 6: Key Takeaways

Congratulations! You've completed Lesson 1 of the ADK tutorial series. Let's review what you learned:


1. **ADK Architecture**
   - `LlmAgent`: The AI brain powered by OpenAI models
   - `Runner`: Manages execution and message flow
   - `SessionService`: Maintains conversation context and memory

2. **Configuration**
   - Set up OpenAI API authentication
   - Used environment variables for API keys
   - Configured the `gpt-5-nano` model

3. **Agent Creation**
   - Defined agent personality through instructions
   - Created different agent personas for different use cases
   - Understood how instructions shape agent behavior

4. **Agent Interaction**
   - Sent messages using the `types.Content` format
   - Processed event streams to get responses
   - Managed sessions to maintain conversation context

### What's Coming Next 🚀

In the upcoming lessons, you'll learn:

- **Lesson 2: Adding Tools to Your Agent**
  - Create custom functions (tools) that agents can call
  - Build a ticket creation system
  - Implement knowledge base search functionality

- **Lesson 3: Multi-Agent Systems**
  - Create specialized agents that work together
  - Build an agent team (triage → specialist → resolution)
  - Implement agent-to-agent communication

- **Lesson 4: Advanced Sessions & State Management**
  - Use persistent storage (Firestore)
  - Implement custom session variables
  - Build long-term memory systems

- **Lesson 5: Integration with MCP**
  - Connect ADK agents to MCP servers
  - Use external tools and data sources
  - Build production-ready agentic systems

### Best Practices Learned 💡

1. **Clear Instructions**: Well-written system prompts are crucial for agent behavior
2. **Session Management**: Use consistent session IDs to maintain conversation context
3. **Error Handling**: Always validate API keys and handle missing configurations
4. **Modularity**: Create reusable helper functions for common operations

### Resources 📚

- [ADK Official Documentation](https://google.github.io/adk-docs/)
- [OpenAI API Documentation](https://platform.openai.com/docs/)
- [ADK Python Repository](https://github.com/google/adk-python)
- [Google Codelabs - ADK Tutorials](https://codelabs.developers.google.com/)

